# TAS4K8s on Minikube

* Minikube using K8s 1.16.7 on Ubuntu 20.04
* Runs within 4 cores and 8GB RAM, increase RAM based on services and app deployments
* Push app, scale
* Service broker with Minibroker

In [ ]:
#env
export TAS_ROOT=~/Downloads/tas-k8s-beta-workshop/tanzu-application-service
export TAS_CONFIG_DIR=~/Downloads/tas-k8s-beta-workshop/configuration-values
export TAS_ARCHIVE=~/Downloads/tanzu-application-service.0.1.0-build.252.tar
mkdir -p $TAS_ROOT $TAS_CONFIG_DIR
cd $TAS_ROOT

In [ ]:
#set minikube default driver
minikube config set driver docker

In [ ]:
minikube status

In [ ]:
#create minikube cluster. choosing latest version of K8s as supported by TKGI
#note: need at least 6 cpus otherwise istio-telemetry would fail to create.
#Ensure remove-resources.yml during TAS deployment is in the right folder
minikube start --cpus=4 --memory=12g --kubernetes-version=1.16.7 --driver=docker 
#--container-runtime=containerd #pks 1.6.2 = k8s 1.15.10, 1.7.0 = k8s 1.16.7

In [ ]:
kubectl config use-context minikube

# Deploy TAS

In [ ]:
#extract TAS downloaded archive
tar -C $TAS_ROOT -xvf $TAS_ARCHIVE 

In [ ]:
#remove resource requirements
curl https://raw.githubusercontent.com/cloudfoundry/cf-for-k8s/master/config-optional/remove-resource-requirements.yml > $TAS_ROOT/custom-overlays/remove-resource-requirements.yml

In [ ]:
#Do this if you want to allow istio ingress to use an IaaS provided LB. Yes, minikube supports it.
#Don't forget to run "minikube tunnel" as tas is being deployed
mv ./custom-overlays/replace-loadbalancer-with-clusterip.yaml ./config-optional/

## Generate config values

In [ ]:
./bin/generate-values.sh -d "vcap.me" > ../configuration-values/deployment-values.yml

Add entries for Pivnet registry and your own image registry (place at end of file, or in two separate files, system-registry-values.yml and app-registry-values.yml):
```
system_registry:
  hostname: registry.pivotal.io
  username: pivnet-username
  password: password

app_registry:
  hostname: https://index.docker.io/v1/
  repository: "username"
  username: "uername"
  password: "password"
```

In [ ]:
#Copy my secrets from pre-existing location
cp ~/Documents/app-registry-values.yml $TAS_CONFIG_DIR
cp ~/Documents/system-registry-values.yml $TAS_CONFIG_DIR

### Ready to deploy TAS. Run minikube load balancer
Run this in another shell as it is a blocking call and requires permission. TAS deployment will not complete unless this is available. It can be run after deployment starts.

```minikube tunnel```

In [ ]:
#deploy tas
./bin/install-tas.sh ../configuration-values

### Configure DNS
Point dns wildcard to placehoder domain to resolve to minikube instance IP


In [ ]:
minikube ip

# Deploy some stuff

In [ ]:
cf api --skip-ssl-validation https://api.vcap.me

In [ ]:
cf auth admin "$(bosh interpolate ../configuration-values/deployment-values.yml --path /cf_admin_password)"

In [ ]:
#needed in beta for now
cf enable-feature-flag diego_docker

In [ ]:
cf create-org test-org
cf create-space -o test-org test-space
cf target -o test-org -s test-space

In [ ]:
#push sample application
cf push demo-app -o rseroter/simple-k8s-app-kpack

## Service Broker with Minibroker

In [ ]:
kubectl create ns minibroker

In [ ]:
helm repo add minibroker https://minibroker.blob.core.windows.net/charts

In [ ]:
helm repo update

In [ ]:
#install into desired namespace. not using service catalog because we will use TAS marketplace
helm install minibroker --namespace minibroker minibroker/minibroker \
  --set "deployServiceCatalog=false" --set "defaultNamespace=minibroker"

In [ ]:
kubectl get all --namespace minibroker

In [ ]:
cf create-service-broker minibroker user pass http://minibroker-minibroker.minibroker.svc.cluster.local

In [ ]:
cf service-brokers

In [ ]:
#see all the services the broker provides
cf service-access

In [ ]:
cf enable-service-access redis
cf enable-service-access mongodb
cf enable-service-access mariadb

In [ ]:
cf marketplace

In [ ]:
cf create-service redis 5-0-7 redis-svc

In [ ]:
cf create-service mariadb 10-3-22 mariadb-svc -c '{"db": {"name": "my_database"}}'

In [ ]:
cf services

In [ ]:
kubectl get pods --namespace minibroker

In [ ]:
kubectl get svc --namespace minibroker

### Test mariadb service
Edit mariadb service - change from ClusterIP to NodePort

```
kubectl port-forward <pod name of master> 3306:3306 --namespace minibroker &
```

In [ ]:
cf create-service-key mariadb-svc key

In [ ]:
cf service-key mariadb-svc key

### Connect to MariaDB through the port forward on your host

```mysql -h localhost --protocol=TCP -u root -p<password from service key>```

### Bind service to an app

* dbbaskette/todos-ui
* cf push todos-app
* edit manifest. Add services: mariadb-svc
* 

# Tear down environment

In [ ]:
#remove cf namespace from cluster
kapp delete -a cf -y

In [ ]:
#stop minikube cluster
minikube stop

In [ ]:
#delete minikube cluster - clears state
minikube delete

In [ ]:
#delete all local clusters and profiles
minikube delete --all